In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from sklearn import preprocessing

In [2]:
df1=pd.read_csv('final_dataset.csv')
df2=pd.read_csv('user_df.csv')

In [8]:
df1.isnull().sum()

Unnamed: 0       0
undergra      3464
from            79
zipcode       1064
iid              0
pid             10
gender           0
dec_o            0
match            0
field_cd        82
age             95
age_o          104
race            63
race_o          73
imprace         79
imprelig        79
goal            79
career_c       138
pf_o_att        89
pf_o_sin        89
pf_o_int        89
pf_o_fun        98
pf_o_amb       107
pf_o_sha       129
dtype: int64

Pour le dataset des données des sujets/utilisateurs, j'ai choisi les features 'iid', 'pid', 'gender', 'age', 'from', 'zipcode', 'imprace', 'match' et 'dec_u'. J'ai décidé d'inclure la colonne 'pid' ici car c'est la colonne du partenaire avec lequel l'utilisateur a interagi, donc j'ai trouvé raisonnable d'ajouter cette colonne dans cet ensemble de données. Je pense que les colonnes 'zipcode' et 'from' sont à peu près les mêmes, et comme mentionné dans l'exercice, je peux supprimer les colonnes qui me semblent similaires. Je ne prendrai donc pas la colonne 'zip' dans cet ensemble de données car elle a plus de données manquantes que dans la colonne 'from' comme indiqué dans la cellule ci-dessus. 

In [9]:
user_df=df1[['iid','pid','gender','age','from','imprace','match']].copy()
user_df

,iid,pid,gender,age,from,imprace,match
0,1,11.0,0,-1.502463,Chicago,-0.627188,0
1,1,12.0,0,-1.502463,Chicago,-0.627188,0
2,1,13.0,0,-1.502463,Chicago,-0.627188,1
3,1,14.0,0,-1.502463,Chicago,-0.627188,1
4,1,15.0,0,-1.502463,Chicago,-0.627188,1
...,...,...,...,...,...,...,...
8373,552,526.0,1,-0.380998,France,-0.978594,0
8374,552,527.0,1,-0.380998,France,-0.978594,0
8375,552,528.0,1,-0.380998,France,-0.978594,0
8376,552,529.0,1,-0.380998,France,-0.978594,0


In [11]:
dec_u=df2[['dec_u']]
user_df.insert(7, "dec_u", dec_u) #insert the dec_u col 
user_df

,iid,pid,gender,age,from,imprace,match,dec_u
0,1,11.0,0,-1.502463,Chicago,-0.627188,0,1
1,1,12.0,0,-1.502463,Chicago,-0.627188,0,1
2,1,13.0,0,-1.502463,Chicago,-0.627188,1,1
3,1,14.0,0,-1.502463,Chicago,-0.627188,1,1
4,1,15.0,0,-1.502463,Chicago,-0.627188,1,1
...,...,...,...,...,...,...,...,...
8373,552,526.0,1,-0.380998,France,-0.978594,0,0
8374,552,527.0,1,-0.380998,France,-0.978594,0,1
8375,552,528.0,1,-0.380998,France,-0.978594,0,1
8376,552,529.0,1,-0.380998,France,-0.978594,0,0


Ensuite, je crée l'ensemble de données des partenaires avec les caractéristiques 'pid', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', qui sont toutes spécifiques aux partenaires et peuvent donc être utilisées comme caractéristiques des partenaires.

In [6]:
partner_df=df1[['pid','pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun','pf_o_amb', 'pf_o_sha']]
partner_df

,pid,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha
0,11.0,0.435433,-0.140833,-0.135311,0.372768,-1.148772,-1.071878
1,12.0,1.969294,-1.791148,-2.547788,3.263458,-1.148772,-1.804465
2,13.0,-0.546238,-0.305864,-0.255935,0.083699,-0.053893,-0.046256
3,14.0,0.128661,-1.378569,-0.738430,3.263458,-0.757744,-1.071878
4,15.0,0.128661,-0.965990,-0.135311,-1.072577,-0.366716,1.125883
...,...,...,...,...,...,...,...
8373,526.0,-1.098428,-0.965990,1.070928,0.372768,-0.366716,0.393296
8374,527.0,1.355749,-0.140833,-1.341550,-1.795249,-0.366716,-1.071878
8375,528.0,0.742205,-0.965990,1.070928,-1.072577,-0.366716,NaN
8376,529.0,-1.098428,0.271746,0.467809,-1.072577,-0.366716,1.125883


Dans mon jeu de données d'utilisateurs, j'ai la colonne 'from' qui contient des données catégorielles. 


In [14]:
user_df['from'].nunique()

269

Nous avons 269 noms de villes dans la colonne 'from', ce qui va créer trop de colonnes si j'utilise l'encodeur onehot ou l'encodeur binaire. J'utiliserais donc Label Encoder et standardiserais les valeurs pour éviter un biais vers cette colonne.

In [13]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
user_df["new_from"] = lb_make.fit_transform(user_df["from"])
user_df[["from", "new_from"]].head(50)

,from,new_from
0,Chicago,49
1,Chicago,49
2,Chicago,49
3,Chicago,49
4,Chicago,49
5,Chicago,49
6,Chicago,49
7,Chicago,49
8,Chicago,49
9,Chicago,49


In [16]:
user_df['std_from'] = (user_df['new_from'] -user_df['new_from'].mean()) / user_df['new_from'].std()
user_df.head()

,iid,pid,gender,age,from,imprace,match,dec_u,new_from,std_from
0,1,11.0,0,-1.502463,Chicago,-0.627188,0,1,49,-1.09769
1,1,12.0,0,-1.502463,Chicago,-0.627188,0,1,49,-1.09769
2,1,13.0,0,-1.502463,Chicago,-0.627188,1,1,49,-1.09769
3,1,14.0,0,-1.502463,Chicago,-0.627188,1,1,49,-1.09769
4,1,15.0,0,-1.502463,Chicago,-0.627188,1,1,49,-1.09769


Maintenant, je peux supprimer les colonnes 'from' et 'new_from'.

In [18]:
final_user_df=user_df.drop(axis=1, columns=['from','new_from'])
final_user_df

,iid,pid,gender,age,imprace,match,dec_u,std_from
0,1,11.0,0,-1.502463,-0.627188,0,1,-1.097690
1,1,12.0,0,-1.502463,-0.627188,0,1,-1.097690
2,1,13.0,0,-1.502463,-0.627188,1,1,-1.097690
3,1,14.0,0,-1.502463,-0.627188,1,1,-1.097690
4,1,15.0,0,-1.502463,-0.627188,1,1,-1.097690
...,...,...,...,...,...,...,...,...
8373,552,526.0,1,-0.380998,-0.978594,0,0,-0.798292
8374,552,527.0,1,-0.380998,-0.978594,0,1,-0.798292
8375,552,528.0,1,-0.380998,-0.978594,0,1,-0.798292
8376,552,529.0,1,-0.380998,-0.978594,0,0,-0.798292


Je pense maintenant que les deux ensembles de données sont prêts à être utilisés par le système de recommandation.

In [19]:
final_user_df.to_csv('prepared_user_df.csv')
partner_df.to_csv('prepared_partners_df.csv')